In [1]:
#Getting required Libraries
import selenium
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import re
import json
from selenium import webdriver
from time import sleep 
from selenium.webdriver.common.by import By

In [2]:
#Creating a webdriver object
driver = webdriver.Chrome()

In [3]:
#Getting the website
driver.get("https://www.zillow.com/mortgage-rates/")
sleep(5)

In [4]:
#Checking for available tables
for table in pd.read_html(driver.page_source):
    print(table)

           Date & Time;       Percent Interest Rate;  \
0      on May 3rd, 2023  the rate was 5.914 percent,   
1      on May 4th, 2023  the rate was 5.711 percent,   
2      on May 5th, 2023  the rate was 5.847 percent,   
3      on May 6th, 2023  the rate was 5.895 percent,   
4      on May 7th, 2023  the rate was 5.903 percent,   
..                  ...                          ...   
86   on July 28th, 2023  the rate was 6.412 percent,   
87   on July 29th, 2023  the rate was 6.376 percent,   
88   on July 30th, 2023  the rate was 6.392 percent,   
89   on July 31st, 2023  the rate was 6.376 percent,   
90  on August 1st, 2023  the rate was 6.804 percent,   

                              Annual Percentage Rate;  
0   and the annual percentage rate was 7.052 percent.  
1   and the annual percentage rate was 6.976 percent.  
2   and the annual percentage rate was 7.025 percent.  
3   and the annual percentage rate was 7.049 percent.  
4   and the annual percentage rate was 7.049 pe

In [5]:
#Getting the Date of Scrape
Curr_date = datetime.datetime.now()
date= Curr_date.date()
date

datetime.date(2023, 8, 2)

In [6]:
#Finding the data in the Conforming loans table
Conforming_loans_table = pd.read_html(driver.page_source)[3]
Conforming_loans_table['Date_of_Scrape'] = date
Conforming_loans_table['Loan type'] = 'conforming_loan'
Conforming_loans_table

,PROGRAM,RATE,1W CHANGE,APR,1W CHANGE.1,Date_of_Scrape,Loan type
0,30-Year Fixed Rate,6.84 %,Arrow Upup0.22 %,6.92 %,Arrow Upup0.22 %,2023-08-02,conforming_loan
1,20-Year Fixed Rate,6.51 %,Arrow Upup0.21 %,6.62 %,Arrow Upup0.21 %,2023-08-02,conforming_loan
2,15-Year Fixed Rate,6.00 %,Arrow Upup0.15 %,6.15 %,Arrow Upup0.16 %,2023-08-02,conforming_loan
3,10-Year Fixed Rate,6.00 %,Arrow Upup0.23 %,6.24 %,Arrow Upup0.25 %,2023-08-02,conforming_loan
4,7-year ARM,7.01 %,Arrow Upup0.39 %,7.61 %,Arrow Upup0.17 %,2023-08-02,conforming_loan
5,5-year ARM,6.80 %,Arrow Upup0.32 %,7.63 %,Arrow Upup0.12 %,2023-08-02,conforming_loan
6,3-year ARM,--,0.00 %,--,0.00 %,2023-08-02,conforming_loan


In [7]:
#Finding the data in the Goverment loans table
Government_loans_table = pd.read_html(driver.page_source)[4]
Government_loans_table['Date_of_Scrape'] = date
Government_loans_table['Loan type'] = 'government_loan'
Government_loans_table

,PROGRAM,RATE,1W CHANGE,APR,1W CHANGE.1,Date_of_Scrape,Loan type
0,30-Year Fixed Rate FHA,6.17 %,Arrow Upup0.31 %,7.29 %,Arrow Upup0.33 %,2023-08-02,government_loan
1,30-Year Fixed Rate VA,6.14 %,Arrow Upup0.07 %,6.43 %,Arrow Upup0.09 %,2023-08-02,government_loan
2,15-Year Fixed Rate FHA,5.82 %,Arrow Upup0.35 %,7.07 %,Arrow Upup0.43 %,2023-08-02,government_loan
3,15-Year Fixed Rate VA,5.84 %,Arrow Upup0.11 %,6.38 %,Arrow Upup0.13 %,2023-08-02,government_loan


In [8]:
#Finding the data in the Jumbo loans table
Jumbo_loans_table = pd.read_html(driver.page_source)[5]
Jumbo_loans_table['Date_of_Scrape'] = date
Jumbo_loans_table['Loan type'] = 'jumbo_loan'
Jumbo_loans_table

,PROGRAM,RATE,1W CHANGE,APR,1W CHANGE.1,Date_of_Scrape,Loan type
0,30-Year Fixed Rate Jumbo,7.33 %,Arrow Upup0.27 %,7.39 %,Arrow Upup0.26 %,2023-08-02,jumbo_loan
1,15-Year Fixed Rate Jumbo,6.91 %,Arrow Upup0.17 %,7.04 %,Arrow Upup0.15 %,2023-08-02,jumbo_loan
2,7-year ARM Jumbo,7.46 %,Arrow Upup0.17 %,7.81 %,Arrow Upup0.08 %,2023-08-02,jumbo_loan
3,5-year ARM Jumbo,7.34 %,Arrow Upup0.17 %,7.76 %,Arrow Upup0.08 %,2023-08-02,jumbo_loan
4,3-year ARM Jumbo,5.72 %,Arrow Upup0.01 %,7.40 %,0.00 %,2023-08-02,jumbo_loan


In [9]:
#Stacking the data
data_stack = pd.concat([Conforming_loans_table,Government_loans_table,Jumbo_loans_table],axis=0,ignore_index=True)
data_stack

,PROGRAM,RATE,1W CHANGE,APR,1W CHANGE.1,Date_of_Scrape,Loan type
0,30-Year Fixed Rate,6.84 %,Arrow Upup0.22 %,6.92 %,Arrow Upup0.22 %,2023-08-02,conforming_loan
1,20-Year Fixed Rate,6.51 %,Arrow Upup0.21 %,6.62 %,Arrow Upup0.21 %,2023-08-02,conforming_loan
2,15-Year Fixed Rate,6.00 %,Arrow Upup0.15 %,6.15 %,Arrow Upup0.16 %,2023-08-02,conforming_loan
3,10-Year Fixed Rate,6.00 %,Arrow Upup0.23 %,6.24 %,Arrow Upup0.25 %,2023-08-02,conforming_loan
4,7-year ARM,7.01 %,Arrow Upup0.39 %,7.61 %,Arrow Upup0.17 %,2023-08-02,conforming_loan
5,5-year ARM,6.80 %,Arrow Upup0.32 %,7.63 %,Arrow Upup0.12 %,2023-08-02,conforming_loan
6,3-year ARM,--,0.00 %,--,0.00 %,2023-08-02,conforming_loan
7,30-Year Fixed Rate FHA,6.17 %,Arrow Upup0.31 %,7.29 %,Arrow Upup0.33 %,2023-08-02,government_loan
8,30-Year Fixed Rate VA,6.14 %,Arrow Upup0.07 %,6.43 %,Arrow Upup0.09 %,2023-08-02,government_loan
9,15-Year Fixed Rate FHA,5.82 %,Arrow Upup0.35 %,7.07 %,Arrow Upup0.43 %,2023-08-02,government_loan
